Начальная популяция - Случайная генерация<br>
Отбор - 20% приспособленных<br>
Мутация - инвертирование битов<br>
Скрещивание - однородное (каждый бит от случайно выбранного родителя)<br>
Новая популяция - замена своих родителей

In [54]:
from numpy import genfromtxt
import random
from random import choice, randint

In [55]:
def get_data():
    with open('5.txt') as f:
        settings = f.readline().split(' ')
    max_weight = int(settings[0])
    max_volume = int(settings[1])
    data = genfromtxt('5.txt', delimiter=' ', dtype=(int, float, int), skip_header=1)
    return {
        'max_weight': int(max_weight),
        'max_volume': float(max_volume),
        'items': data.tolist()
    }

data = get_data()

items = sorted(data['items'], key=lambda x: x[2], reverse=True)

max_w = data['max_weight']
max_v = data['max_volume']
print(max_w, max_v)

n = 200
generations = 500

13000 12.0


Начальная популяция - Случайная генерация

In [56]:
def generate(item):
    individ = ""
    wei = val = vol = 0
    for i in range(len(item)):
        ind = random.randint(0, 1)
        if (ind):
            wei += item[i][0]
            vol += item[i][1]
            val += item[i][2]
            
        individ += str(ind)
    return {"gen": individ, "wei": wei, "vol": vol, "val": val}

population = []
for i in range(n):
    population.append(generate(items))
print(len(population))

200


Отбор - 20% приспособленных

In [57]:
def fitness(ind):
    summ_val = summ_vol = summ_wei = 0
    for i in range(len(items)):
        summ_wei += float(ind[i])*items[i][0]
        summ_vol += float(ind[i])*items[i][1]
        summ_val += float(ind[i])*items[i][2]
        if summ_vol<=max_v and summ_wei<=max_w:
            price = summ_val
        else:
            price=0
    return price


def selection(population, s=20):
    percent = int(len(population) * s / 100) 
    population = sorted(population, key = lambda i: i['val'],reverse=True)
    
    select = []
    not_select = []
    count = 0
                      
    for i in population:
        if (fitness(i['gen']) == 0):
            not_select.append(i)
        elif(count < percent):
            select.append(i)
            count += 1
        else:
            not_select.append(i)            
    return  select,not_select


Мутация - инвертирование битов

In [58]:
def mutate(ind):
    newChromo = ''
    for i in ind:
        if i=='0':
            gen = '1'
        else:
            gen = '0'
        newChromo += gen
    return {'gen': newChromo, 'wei': 0,  'vol': 0, 'val':0}

Скрещивание (кроссинговер) между выбранными особями. Каждая особь скрещивается 1 раз за 1 поколение, 1 пара дает 2 потомка : однородный (каждый бит от случайно выбранного родителя)

In [59]:
def crossingover(parent1, parent2):
    parents = [parent1, parent2]
    childs = []
    for c in range(2):
        mask = []
        for i in range(len(parent1)):
            mask.append(random.randrange(2))
        child = ''
        j = 0
        for i in mask:
            child += parents[i][j]
            j += 1
        childs.append(child)
    return childs

def getOffspring(indToCross):
    offspring =[]
    while indToCross!=[]:
        ind1 = choice(indToCross)
        indToCross.remove(ind1)
        ind2 = choice(indToCross)
        indToCross.remove(ind2)
        childs= crossingover(ind1['gen'], ind2['gen'])
        offspring.append({'gen': childs[0], 'wei': 0,  'vol': 0, 'val':0})
        offspring.append({'gen': childs[1], 'wei': 0,  'vol': 0, 'val':0})
    mutInd = choice(offspring)
    offspring.remove(mutInd)
    offspring.append(mutate(mutInd['gen']))
    return offspring

Новая популяция - замена своих родителей

In [60]:
def calcFitness(ind):
    summ_val = summ_vol = summ_wei = 0
    for i in range(len(items)):
        summ_wei += float(ind[i])*items[i][0]
        summ_vol += float(ind[i])*items[i][1]
        summ_val += float(ind[i])*items[i][2]
        if summ_vol<=max_v and summ_wei<=max_w:
            price = summ_val
        else:
            price=0
    
    return {'gen': ind, 'wei': summ_wei, 'vol': summ_vol, 'val': price}

In [61]:
for i in range(generations):
    
    sel, not_sel  = selection(population)
    offspring = getOffspring(sel)
   
    for i in offspring:
        not_sel.append(calcFitness(i["gen"]))
    
sel, not_sel = selection(population)
population = sorted(sel, key = lambda i: i['val'],reverse=True)
print(population[0]['val'])
print(population[0]['vol'])
print(population[0]['wei'])

3867
12.0
11649


In [62]:
for i in population:
    print(i)

{'gen': '011111100000011110011000001001', 'wei': 11649, 'vol': 12.0, 'val': 3867}
{'gen': '111001101010000101000110011010', 'wei': 12994, 'vol': 11.6, 'val': 3739}
{'gen': '111110000101100010000001011101', 'wei': 12520, 'vol': 11.7, 'val': 3704}
{'gen': '001101101011101000000011011010', 'wei': 12811, 'vol': 11.6, 'val': 3675}
{'gen': '100110010111100000100111000010', 'wei': 10393, 'vol': 11.899999999999999, 'val': 3516}
{'gen': '010010110111101000010010000101', 'wei': 11800, 'vol': 10.0, 'val': 3515}
{'gen': '011000010011011101010010001101', 'wei': 11019, 'vol': 10.999999999999998, 'val': 3482}
{'gen': '010100110011000100110011100101', 'wei': 10930, 'vol': 11.700000000000001, 'val': 3447}
{'gen': '100111001101000110011010000000', 'wei': 8675, 'vol': 11.499999999999998, 'val': 3434}
{'gen': '110100100010010101101100100100', 'wei': 11980, 'vol': 11.7, 'val': 3432}
{'gen': '101101100001000100100110110100', 'wei': 11737, 'vol': 11.1, 'val': 3376}
{'gen': '101110010100011100000000101010', '